In [ ]:
from collections import defaultdict
import csv 
from csv import reader
from scipy.optimize import linear_sum_assignment
#from PIL import Image as im
from IPython.core.display import Image, display
import requests
from io import BytesIO
import json 

'''
Ambiguity
    Multiple Options > A/M
    Definition, Location > A/L
    Definition, Destination/Intention > A/D
    Definition, Kind/Type > A/K
    Definition, Time > A/T
    Gouping > A/G

Uncertainty > U

Mistakes/Bad Data
    Annotator mistakes > M/A
    Bad question/bad images > M/B

Misc/need reclassifying > ~

Question with multiple classification will have '.' between the classificiations
ex: A/M.A/L.M/B)
'''

input_csv = "csv_results_ambiguity.csv"
data = []
with open(input_csv) as read_obj:
    csv_reader = csv.DictReader(read_obj)
    for row in csv_reader:
        data.append(row)


In [ ]:
'''
Display by classification
'''
search_item = 'A/L'

for row in data:
    temp = row['Answer.skip_reason'].strip('"')
    labels = temp.split('.')
    
    if search_item in labels:
        groups = []

        new_questions = json.loads(row['Answer.answer_questions'])
        for i, g in enumerate(json.loads(row['Answer.answer_groups'])):
            group = []
            new_question = new_questions[i]
            for x in g:
                group.append(x['content'])
            groups.append({new_question: group})
        print(row['Input.questionStr'])
        print(row['Input.imgUrl'])
        print(labels)

        image_url = row['Input.imgUrl'].strip('"')
        display(Image(url=image_url))
       

In [ ]:
'''
Display by number of classifications
'''
search_num = 3

for row in data:
    temp = row['Answer.skip_reason'].strip('"')
    labels = temp.split('.')
    
    if len(labels) >= search_num:
        groups = []

        new_questions = json.loads(row['Answer.answer_questions'])
        for i, g in enumerate(json.loads(row['Answer.answer_groups'])):
            group = []
            new_question = new_questions[i]
            for x in g:
                group.append(x['content'])
            groups.append({new_question: group})
        print(row['Input.questionStr'])
        print(row['Input.imgUrl'])
        print(groups)
        print(labels)

        image_url = row['Input.imgUrl'].strip('"')
        display(Image(url=image_url))

In [ ]:
import seaborn as sns 
from matplotlib import pyplot as plt 
import numpy as np 

num_mistakes_other = 0
num_to_keep = 0
total = 0
# histogram 
acceptable = ['A/M', 'A/G', 'A/D', 'A/K', 'A/T', 'U', 'M/A', 'M/B', 'skip', '~']
to_keep = ['A/M', 'A/G', 'A/D', 'A/K', 'A/T',  '~']
all_labels = []
for row in data:
    temp = row['Answer.skip_reason'].strip('"')
    labels = temp.split('.')
    for lab in labels:
        if lab in acceptable:
            all_labels.append(lab)
    total += 1
    for lab in labels:
        if lab in to_keep: 
            num_to_keep += 1 
            break
        else:
            num_mistakes_other  += 1
            break



g = sns.histplot(all_labels)

plt.xticks(rotation=90)


print(f"There are {total} examples, with {num_to_keep} good ones and {num_mistakes_other} bad ones.")